In [1]:
# model.compile(loss=_loss, optimizer=_optimizer, metrics=[custom_loss_wrapper_2(model.input)])
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split

_activation = Activation('softmax')
_optimizer = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

def custom_loss_wrapper_2(inputs):
    print("inputs {}".format(inputs.shape))
    # source: https://stackoverflow.com/questions/55445712/custom-loss-function-in-keras-based-on-the-input-data
    # 2nd source: http://stackoverflow.com/questions.55597335/how-to-use-tf-gather-in-batch
    def reindex(tensor_tuple):
        # unpack tensor tuple
        y_true = tensor_tuple[0]
        y_pred = tensor_tuple[1]
        t_inputs = K.cast(tensor_tuple[2], dtype='int64')
        t_max_indices = K.tf.where(K.tf.equal(t_inputs, K.max(t_inputs)))

        # gather the values from y_true and y_pred
        print("y_true {}".format(y_true.shape))
        print("y_pred {}".format(y_pred.shape))
        y_true_gathered = K.gather(y_true, t_max_indices)
        y_pred_gathered = K.gather(y_pred, t_max_indices)

        print(K.mean(K.square(y_true_gathered - y_pred_gathered)))

        return K.mean(K.square(y_true_gathered - y_pred_gathered))

    def custom_loss(y_true, y_pred):
        print("y_true2 {}".format(y_true.shape))
        print("y_pred2 {}".format(y_pred.shape))

        # Step 1: "tensorize" the previous list
        t_inputs = K.variable(inputs)

        # Step 2: Stack tensors
        tensor_tuple = K.stack([y_true, y_pred, t_inputs], axis=1)

        vals = K.map_fn(reindex, tensor_tuple, dtype='float32')
        print('vals: {}'.format(vals.shape))
        print('kvals: {}'.format(K.mean(vals).shape))
        return K.mean(vals, keepdims=True)

    return custom_loss



In [1]:
"""Convolutional Long-Short Term Model.
"""
import os, sys
import glob
import numpy as np

from sclouds.helpers import get_lon_array, get_lat_array, path_convlstm_results
from sclouds.ml.ConvLSTM.utils import r2_keras

from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

#my_callbacks = [
    #tf.keras.callbacks.EarlyStopping(patience=2),
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
#]
#model.fit(dataset, epochs=10, callbacks=my_callbacks)

class ConvLSTM:
    """ A convoliutional lstm neural network.

    What about :
        recurrent_activation='hard_sigmoid'
        activation='tanh'

    Notes
    ----------------------------------------------------------------------------
    filters, kernel_size, strides=(1, 1), padding='valid', data_format=None,
    dilation_rate=(1, 1), activation='tanh', recurrent_activation='hard_sigmoid',
    use_bias=True, kernel_initializer='glorot_uniform',
    recurrent_initializer='orthogonal', bias_initializer='zeros',
    unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None,
    bias_regularizer=None, activity_regularizer=None, kernel_constraint=None,
    recurrent_constraint=None, bias_constraint=None, return_sequences=False,
    go_backwards=False, stateful=False, dropout=0.0, recurrent_dropout=0.0

    (x=x, y=y, batch_size=None, epochs=1, verbose=1, callbacks=None,
    validation_split=0.2, validation_data=None, shuffle=True, class_weight=None,
    sample_weight=None, initial_epoch=0, steps_per_epoch=None,
    validation_steps=None, validation_batch_size=None, validation_freq=1,
    max_queue_size=10,

    """

    DATA_FORMAT        = 'channels_last'
    PADDING            = 'same'
    RETURN_SEQUENCE    = True
    NUM_INPUT_VARS     = 4
    OUTPUT_KERNEL_SIZE = 1
    OUTPUT_FILTER      = 1
    KERNAL_INIT        = 'lecun_uniform'

    n_lat   = 81
    n_lon   = 161
    WORKERS = 16 # identical to the number of cores requested in

    USE_MULTIPROCESSING = True
    #early_stopping_monitor = EarlyStopping(patience=3)
    #CALLBACKS = [early_stopping_monitor, TensorBoard(log_dir='./logs')]

    def __init__(self, X_train, y_train, filters, kernels, seq_length = 24,
                 epochs=40, batch_size = 20, validation_split=0.1, name = None, result_path = None):

        self.filters = filters
        self.kernels = kernels
        self.seq_length = seq_length

        self.X_train = X_train
        self.y_train = y_train
        self.epochs = epochs
        self.batch_size = batch_size

        self.validation_split = validation_split
        print('Starts to build model ...')
        self.model = self.build_model(filters, kernels, seq_length)
        print('Statrs compilation of model ...')
        self.name = name
        self.result_path = '/home/hanna/'
        self.model.compile(optimizer=keras.optimizers.Adam(
                            learning_rate=0.001,
                            beta_1=0.9,
                            beta_2=0.999,
                            epsilon=1e-07,
                            amsgrad=False,
                            name="Adam",),
                            loss='mean_squared_error',
                            metrics=['mean_squared_error', r2_keras])
        print('starts training')
        self.history = self.model.fit(X_train, y_train, batch_size=batch_size,
                                     epochs=epochs, verbose=1,
                                     #callbacks=self.CALLBACKS,
                                     #validation_split=self.validation_split,
                                     #validation_data=None,
                                     shuffle=False,
                                     #class_weight=None,
                                     #sample_weight=None, initial_epoch=0,
                                     #steps_per_epoch=100,
                                     #validation_steps=None,
                                     #validation_freq=1, max_queue_size=10,
                                     workers=self.WORKERS,
                                     use_multiprocessing= self.USE_MULTIPROCESSING)
        self.store_history()
        self.store_summary()
        print('finished model -- ')


    def build_model(self, filters, kernels, seq_length = 24):
        """" Building a ConvLSTM model for predicting cloud cover.
        All filters are squared. Adding the architecture.

        Parameteres
        ------------------------
        filters : array like
            use length of this to infer the depth of the network.

        Returns
        ------------------------
        model : tensorflow.keras.Sequential
            Builded model
        """


        model =  keras.Sequential()

        model.add( keras.layers.Input(batch_input_shape=(self.batch_size, seq_length, self.n_lat, self.n_lon,
                                self.NUM_INPUT_VARS), name='input'))        #batch_size = self.batch_size)

        # Adding the first layer
        model.add(keras.layers.ConvLSTM2D(filters = filters[0],
                           kernel_size = (kernels[0], kernels[0]), #, self.NUM_INPUT_VARS
                           input_shape = (seq_length,
                                            self.n_lat, self.n_lon, self.NUM_INPUT_VARS),
                           kernel_initializer=self.KERNAL_INIT,
                           padding = self.PADDING,
                           return_sequences=self.RETURN_SEQUENCE,
                           data_format=self.DATA_FORMAT,
                           batch_size = self.batch_size))

        prev_filter = filters[0]
        if len(filters) > 1 and len(kernels) > 1:
            print('Detected more than one layer ... ')
            for i, tuple in enumerate(zip(filters[1:], kernels[1:])):
                filter, kernal = tuple
                # Begin with 3D convolutional LSTM layer
                model.add(keras.layers.ConvLSTM2D(filters=filter,
                                                kernel_size=(kernal, kernal), # prev_filter
                                                input_shape = (seq_length, self.n_lat,
                                                                self.n_lon, prev_filter),
                                                kernel_initializer=self.KERNAL_INIT,
                                                padding = self.PADDING,
                                                return_sequences=self.RETURN_SEQUENCE,
                                                data_format=self.DATA_FORMAT,
                                                batch_size = self.batch_size))
                prev_filter = filter
        # Adding the last layer
        model.add(keras.layers.ConvLSTM2D(filters=self.OUTPUT_FILTER,
                                        kernel_size=(self.OUTPUT_KERNEL_SIZE, self.OUTPUT_KERNEL_SIZE), #prev_filter
                                        input_shape = (seq_length, self.n_lat,
                                                        self.n_lon, prev_filter),
                                        kernel_initializer=self.KERNAL_INIT,
                                        padding = self.PADDING,
                                        return_sequences=self.RETURN_SEQUENCE,
                                        data_format=self.DATA_FORMAT,
                                        batch_size = self.batch_size))

        model.summary()
        return model

    def compile(self, lmd=0.001):
        """ Compile model.

        Parameters
        -------------
        model : tensorflow.keras.Sequential
            Build model.

        Returnes
        -------------
        model : tensorflow.keras.Sequential
            Compiled model.
        """
        _loss = custom_loss_wrapper_2(self.model.inputs)
        self.model.compile(optimizer=keras.optimizers.Adam(
                            learning_rate=lmd,
                            beta_1=0.9,
                            beta_2=0.999,
                            epsilon=1e-07,
                            amsgrad=False,
                            name="Adam",),
                            loss= _loss,
                            metrics= [_loss])
        return self.model



    def store_history(self):
        """ Fit builded model.
        Parameters
        -------------
        model : tensorflow.keras.Sequential
            Builded model
        """
        import pandas as pd
        history = self.history

        # convert the history.history dict to a pandas DataFrame:
        hist_df = pd.DataFrame(history.history)

        # save to json:
        hist_json_file = os.path.join(self.result_path, 'history.json')
        with open(hist_json_file, mode='w') as f:
            hist_df.to_json(f)

        # or save to csv:
        hist_csv_file = os.path.join(self.result_path, 'history.csv')
        with open(hist_csv_file, mode='w') as f:
            hist_df.to_csv(f)

        return

    def store_summary(self):
        """ Store summary of tranings process.
        """
        ORIG_OUTPUT = sys.stdout
        with open(os.path.join(self.result_path, "summary_{}.txt".format(self.name)), "w") as text_file:
            sys.stdout = text_file
            self.model.summary()
        sys.stdout = ORIG_OUTPUT
        self.model.save(os.path.join(self.result_path,'{}.h5'.format(self.name)))  # creates a HDF5 file 'my_model.h5'
        return


Warning.. Using pgf backend, no GUI available. use plt.savefig() for inpection


In [2]:
import tensorflow as tf
num_vars = 4
# (seq_length, self.n_lat, self.n_lon, self.NUM_INPUT_VARS),
seq_length = 24

epochs = 40
batch_size = 2 #20
dummy_num_samples = 50
X_train = tf.ones((batch_size*2, seq_length, 81, 161, num_vars))
y_train = tf.ones((batch_size*2, seq_length, 81, 161))

# antall filrer i hver lag.
filters = [8] #256, 128,
# size of filters used 
kernels = [3] #, 3, 3
#from utils import get_xarray_dataset_for_period, get_data_keras
#data = get_xarray_dataset_for_period(start = '2012-01-01', stop = '2012-01-31')
#print(data)
#X_train, y_train = get_data_keras(data, num_samples = None, seq_length = 24, batch_size = 10,
#                data_format='channels_last')

model = ConvLSTM(X_train=X_train, y_train=y_train, filters=filters,
                 kernels=kernels, seq_length = seq_length,
                 epochs=epochs, batch_size = batch_size, validation_split=0.1,
                 name = 'test_model', result_path = '/home/hannasv/')


Starts to build model ...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (2, 24, 81, 161, 8)       3488      
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (2, 24, 81, 161, 1)       40        
Total params: 3,528
Trainable params: 3,528
Non-trainable params: 0
_________________________________________________________________
Statrs compilation of model ...
starts training
Train on 4 samples
Epoch 1/40
4/4 [==============================] - ETA: 7s - loss: 0.7235 - mean_squared_error: 0.7235 - r2_keras: -4528962797568.00 - 11s 3s/sample - loss: 0.7002 - mean_squared_error: 0.7002 - r2_keras: -4383228035072.0000
Epoch 2/40
4/4 [==============================] - ETA: 2s - loss: 0.6352 - mean_squared_error: 0.6352 - r2_keras: -3976012496896.00 - 6s 2s/sample - loss: 0.6174 - mean_squared_error: 0.6174 - r2_ker

In [ ]:
import numpy as np
batch_size = 9
seq_length = 24
num_vars = 4
y_pred  = np.ones((batch_size*2, seq_length, 81, 161))
y_train = np.ones((batch_size*2, seq_length, 81, 161))

In [15]:
#np.sum(y_pred - y_train)

In [16]:
import tensorflow as tf
num_vars = 4
# (seq_length, self.n_lat, self.n_lon, self.NUM_INPUT_VARS),

seq_length = 24
epochs = 10
batch_size = 3 #20
dummy_num_samples = 50
X_train = tf.ones((batch_size*2, seq_length, 81, 161, num_vars))
y_train = tf.ones((batch_size*2, seq_length, 81, 161))

# antall filrer i hver lag.
filters = [8] #256, 128,
# size of filters used 
kernels = [3] #, 3, 3
#from utils import get_xarray_dataset_for_period, get_data_keras
#data = get_xarray_dataset_for_period(start = '2012-01-01', stop = '2012-01-31')
#print(data)
#X_train, y_train = get_data_keras(data, num_samples = None, seq_length = 24, batch_size = 10,
#                data_format='channels_last')

model = ConvLSTM(X_train=X_train, y_train=y_train, filters=filters,
                 kernels=kernels, seq_length = seq_length,
                 epochs=epochs, batch_size = batch_size, validation_split=0.1,
                 name = 'test_model', result_path = '/home/hannasv/')

Starts to build model ...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_6 (ConvLSTM2D)  (3, 24, 81, 161, 8)       3488      
_________________________________________________________________
conv_lst_m2d_7 (ConvLSTM2D)  (3, 24, 81, 161, 1)       40        
Total params: 3,528
Trainable params: 3,528
Non-trainable params: 0
_________________________________________________________________
Statrs compilation of model ...
starts training
Train on 5 samples, validate on 1 samples
Epoch 1/10
5/5 [==============================] - ETA: 7s - loss: 1.1709 - mean_squared_error: 1.1709 - r2_keras: -10994623447040.000 - 12s 2s/sample - loss: 1.1709 - mean_squared_error: 1.1709 - r2_keras: -10994623447040.0000


InvalidArgumentError:  Specified a list with shape [3,81,161,4] from a tensor with shape [2,81,161,4]
	 [[node sequential_3/conv_lst_m2d_6/TensorArrayUnstack/TensorListFromTensor (defined at <ipython-input-12-d2624e7a0581>:102) ]] [Op:__inference_distributed_function_22498]

Function call stack:
distributed_function
